In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

# Step 1: Load Models
yolo_face_model = YOLO("./models/yolov11n-face.pt").to('cpu')  # YOLO Face detection
classification_model = YOLO("./student_faces/yolo_classification/weights/best.pt").to('cpu')  # YOLO Classification

# Define class labels for YOLO classification
classes = ['245322748034', '245322748042', '245322748045']  # Your student IDs or labels

# Step 2: Main Pipeline Function
def process_image(image_path):
    """
    Pipeline: 
    - Detect faces using YOLO Face Detector
    - Classify each detected face using YOLO Classification Model
    - Display the class predictions with confidence scores on the image
    """
    # Load and prepare the image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 1. Detect Faces using YOLO Face Detector
    print("Running YOLO Face Detection...")
    face_results = yolo_face_model.predict(source=image_path, imgsz=224, conf=0.5)
    face_boxes = face_results[0].boxes.xyxy.cpu().numpy()  # Bounding box predictions

    if len(face_boxes) == 0:
        print("No faces detected in the image.")
        return

    for i, box in enumerate(face_boxes):
        x1, y1, x2, y2 = map(int, box[:4])  # Extract bounding box coordinates
        cropped_face = img_rgb[y1:y2, x1:x2]  # Crop the detected face

        if cropped_face.size == 0:
            print(f"Invalid face region for box {i}.")
            continue

        # Resize face to 224x224 for YOLO classification
        face_resized = cv2.resize(cropped_face, (224, 224))

        # 2. Predict Class using YOLO Classification Model
        print("Running YOLO Classification Model...")
        classification_results = classification_model.predict(source=face_resized, imgsz=224, conf=0.5)

        if hasattr(classification_results[0], 'probs') and classification_results[0].probs is not None:
            class_probs = classification_results[0].probs.data  # Class probabilities
            predicted_class_idx = class_probs.argmax()  # Index of the most confident class
            predicted_class = classes[predicted_class_idx]  # Map to class labels
            confidence_classification = class_probs[predicted_class_idx]
            print(f"Face {i+1} - Class: {predicted_class}, Confidence: {confidence_classification:.2f}")
        else:
            print(f"Face {i+1} - No class predictions returned by YOLO.")
            continue

        # 3. Draw Results on Image
        label = f"Class: {predicted_class} | Confidence: {confidence_classification:.2f}"
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Bounding box
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show Image with Results
    output_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imshow("YOLO Face Detection and Classification", output_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Step 3: Run Pipeline on a Test Image
image_path = "./WhatsApp Image 2024-12-17 at 19.16.08_e2e1dcfa.jpg"  # Replace with your test image path
process_image(image_path)
